# ESM Exercise Selection


# I IMPORT PACKAGES

In [195]:
import pandas as pd
import numpy as np
import os

from pathlib import Path

# II SETING UP DIRECTORIES

In [196]:
os.chdir("G:\Meu Drive\Selection\esm_selection")

root_dir = Path(os.getcwd())
data_dir = Path(root_dir, "Data/")
input_dir = Path(data_dir, "Input/")
output_dir = Path(data_dir, "Output")

# 1 OPEN DATA

In [197]:
database = (pd.read_table(Path(input_dir, "trainingdata.csv"), sep = ",", header= None)
        .rename(columns = {1: "group",
                           0: "data",
                           2: "y"},)
        )

database.describe(include = "all")

,data,group,y
count,4600,4600.000000,4600.000000
unique,230,NaN,NaN
top,01-Oct-2002,NaN,NaN
freq,21,NaN,NaN
mean,NaN,10.500000,31.732664
std,NaN,5.766908,1474.257530
min,NaN,1.000000,6.954679
25%,NaN,5.750000,9.800119
50%,NaN,10.500000,9.997224
75%,NaN,15.250000,10.196649


In [198]:
database.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   data    4600 non-null   object 
 1   group   4600 non-null   int64  
 2   y       4600 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 107.9+ KB


With .info() we can`t see any missing data, but with .describe() we can see there is y value faraway from rest.

# 2 DATA MANIPULATION

In [199]:
data_transformed = (database
                    .pivot(index = ["data"],
                           columns = ["group"],
                           values = ["y"]
                           )
                    )

ValueError: Index contains duplicate entries, cannot reshape

By the error message, there are some duplicated data, so we need to identify where and witch is the correct one.

In [200]:
database.groupby(["group"]).nunique()

,data,y
group,,
1,229,230
2,230,230
3,230,230
4,230,230
5,230,230
6,230,230
7,230,230
8,230,230
9,230,230


The problem is in group one. Wicth data is repeated?

In [201]:
database.query(" group == 1 ").mode()

,data,group,y
0,01-Oct-2002,1.0,8.164459
1,NaN,NaN,8.256282
2,NaN,NaN,8.472554
3,NaN,NaN,8.747516
4,NaN,NaN,8.790984
...,...,...,...
225,NaN,NaN,11.397031
226,NaN,NaN,11.494487
227,NaN,NaN,11.555528
228,NaN,NaN,11.725619


The problem is in "01-Oct-2002". Let`s see what is the right data.

In [202]:
(database
 .groupby(["data"])
 .count()
 .sort_values(["group"])
 .query(" group != 20  ")
)

,group,y
data,,
01-Nov-2002,19,19
01-Oct-2002,21,21


So the problem is in 01-Oct-2002 and one of then should be 01-Nov-2002.

In [203]:
# The right data shod be '01-Nov-2002'
database = (database
            .assign(duplicated = lambda df:  df[["group", "data"]].duplicated(),
                    data       =  lambda df: df["data"].mask(df["duplicated"] == True, '01-Nov-2002')
                    )
            )
database.query(" duplicated == True ")

,data,group,y,duplicated
34,01-Nov-2002,1,10.124778,True


Since there are no duplicated data anymore, we can wide the database. But before that, to avoid numerical labels, we paste a "group_" suffix to the group values.

In [204]:
# Now there are no duplicated data, we can wide the DataFrame
database_wide = (database
                        .assign(group = lambda df: "group_" + df["group"].astype("str") )
                        .pivot(index = ["data"],
                               columns = ["group"],
                               values = "y"
                               )
                        .reset_index()
                        .rename_axis(None, axis = 1)
                        )
database_wide.describe()

,group_1,group_10,group_11,group_12,group_13,group_14,group_15,group_16,group_17,group_18,group_19,group_2,group_20,group_3,group_4,group_5,group_6,group_7,group_8,group_9
count,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000
mean,444.709754,10.048824,9.996743,10.011681,9.958684,9.967945,9.967287,10.045631,9.975825,10.014140,9.970744,10.017845,9.955394,9.997491,9.948381,10.007684,10.062900,10.023976,9.981162,9.991193
std,6593.081092,0.535920,0.485479,0.566335,0.513044,0.562966,0.540261,0.461217,0.495306,0.369347,0.509023,0.508128,0.628132,0.502479,0.516962,0.558968,0.587898,0.605233,0.502275,0.531928
min,8.164459,8.045215,8.132061,7.319235,7.593635,7.891626,7.207311,8.508621,8.496057,8.481863,7.818055,7.843785,6.954679,7.764694,7.725457,7.082428,8.261903,7.898420,8.022692,7.656090
25%,9.721888,9.844656,9.813506,9.818821,9.714076,9.703836,9.794658,9.850055,9.788434,9.840511,9.854594,9.851758,9.702203,9.812865,9.713000,9.820939,9.815503,9.830740,9.855542,9.805584
50%,9.979530,9.996489,9.987516,10.013541,9.994251,9.959631,9.979454,10.025458,9.981834,10.005137,9.999646,9.993396,10.006072,10.024649,9.964654,10.001468,10.021673,10.009085,9.999617,9.993424
75%,10.133512,10.235389,10.201939,10.250564,10.223290,10.181924,10.165320,10.218791,10.162658,10.159051,10.171975,10.162437,10.267470,10.211335,10.192324,10.208402,10.259789,10.238493,10.146274,10.182783
max,99999.000000,12.563542,11.777333,12.171259,12.116019,12.286089,11.894928,11.752984,12.096245,11.310129,11.785191,12.906505,12.250087,12.674424,11.668456,12.583304,13.311029,13.541790,12.414211,13.018325


Now we can solve the outlier problem. We will do a median imputation.

In [206]:
limit = np.quantile(database_wide.drop(["data"], axis = 1), 0.999) # limit to mark as wrong value

database_new = (database_wide
                .assign(group_1_aux = lambda df: df["group_1"].mask( df["group_1"] > limit, np.nan),
                        group_1     = lambda df: df["group_1_aux"].fillna(df["group_1_aux"].median)
                        )
                )
database_new.describe()

,group_10,group_11,group_12,group_13,group_14,group_15,group_16,group_17,group_18,group_19,group_2,group_20,group_3,group_4,group_5,group_6,group_7,group_8,group_9,group_1_aux
count,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,229.000000
mean,10.048824,9.996743,10.011681,9.958684,9.967945,9.967287,10.045631,9.975825,10.014140,9.970744,10.017845,9.955394,9.997491,9.948381,10.007684,10.062900,10.023976,9.981162,9.991193,9.974862
std,0.535920,0.485479,0.566335,0.513044,0.562966,0.540261,0.461217,0.495306,0.369347,0.509023,0.508128,0.628132,0.502479,0.516962,0.558968,0.587898,0.605233,0.502275,0.531928,0.500889
min,8.045215,8.132061,7.319235,7.593635,7.891626,7.207311,8.508621,8.496057,8.481863,7.818055,7.843785,6.954679,7.764694,7.725457,7.082428,8.261903,7.898420,8.022692,7.656090,8.164459
25%,9.844656,9.813506,9.818821,9.714076,9.703836,9.794658,9.850055,9.788434,9.840511,9.854594,9.851758,9.702203,9.812865,9.713000,9.820939,9.815503,9.830740,9.855542,9.805584,9.721064
50%,9.996489,9.987516,10.013541,9.994251,9.959631,9.979454,10.025458,9.981834,10.005137,9.999646,9.993396,10.006072,10.024649,9.964654,10.001468,10.021673,10.009085,9.999617,9.993424,9.979439
75%,10.235389,10.201939,10.250564,10.223290,10.181924,10.165320,10.218791,10.162658,10.159051,10.171975,10.162437,10.267470,10.211335,10.192324,10.208402,10.259789,10.238493,10.146274,10.182783,10.124778
max,12.563542,11.777333,12.171259,12.116019,12.286089,11.894928,11.752984,12.096245,11.310129,11.785191,12.906505,12.250087,12.674424,11.668456,12.583304,13.311029,13.541790,12.414211,13.018325,11.725619


# 3 EXPORT DATA

To finish, we export the new database as .csv and .xlsx, adding years and months variables to facilitate further manipulation. We also export a long version of the new data, that would be more suitable for some applications.

In [209]:
database_new = (database_new
                .assign(year = lambda df: df["data"].str.slice(7,11),
                        month = lambda df: df["data"].str.slice(3,6),
                        )
                )
database_new.head()

,data,group_1,group_10,group_11,group_12,group_13,group_14,group_15,group_16,group_17,...,group_3,group_4,group_5,group_6,group_7,group_8,group_9,group_1_aux,year,month
0,01-Apr-2000,10.01135,9.987146,8.372398,11.204862,10.000080,9.957437,9.366857,10.019017,9.958579,...,9.528391,9.682856,10.021460,10.634633,10.090673,10.032882,9.189847,10.011350,2000,Apr
1,01-Apr-2001,10.77335,10.020888,10.025045,10.342786,9.386033,10.039840,9.895056,9.813690,9.903303,...,9.729937,9.661809,9.998162,10.097819,9.961338,10.108989,10.224766,10.773350,2001,Apr
2,01-Apr-2002,9.379179,8.486581,9.864623,9.487962,10.232624,11.681487,9.978239,9.611069,9.910267,...,9.584221,10.889030,9.982543,9.813211,10.015782,9.535142,8.915741,9.379179,2002,Apr
3,01-Apr-2003,10.017709,9.482085,10.433054,10.060909,9.379115,10.216647,9.792787,9.771448,9.804754,...,10.567198,10.078124,10.175519,9.776505,9.105832,9.939589,10.259827,10.017709,2003,Apr
4,01-Apr-2004,9.821178,9.959817,9.304840,10.553361,9.791334,9.876744,10.003894,9.879274,9.536919,...,12.674424,10.354162,10.071010,9.904600,9.963596,10.352201,10.558290,9.821178,2004,Apr


In [ ]:
database_new#%% md
# ESM Exercise Selection


# I IMPORT PACKAGES

In [ ]:
import pandas as pd
import numpy as np
import os

from pathlib import Path

# II SETING UP DIRECTORIES

In [ ]:
os.chdir("G:\Meu Drive\Selection\esm_selection")

root_dir = Path(os.getcwd())
data_dir = Path(root_dir, "Data/")
input_dir = Path(data_dir, "Input/")
output_dir = Path(data_dir, "Output")

# 1 OPEN DATA

In [ ]:
database = (pd.read_table(Path(input_dir, "trainingdata.csv"), sep = ",", header= None)
        .rename(columns = {1: "group",
                           0: "data",
                           2: "y"},)
        )

database.describe(include = "all")

In [ ]:
database.info()

With .info() we can`t see any missing data, but with .describe() we can see there is y value faraway from rest.

# 2 DATA MANIPULATION

In [ ]:
data_transformed = (database
                    .pivot(index = ["data"],
                           columns = ["group"],
                           values = ["y"]
                           )
                    )

By the error message, there are some duplicated data, so we need to identify where and witch is the correct one.

In [ ]:
database.groupby(["group"]).nunique()

The problem is in group one. Wicth data is repeated?

In [ ]:
database.query(" group == 1 ").mode()

The problem is in "01-Oct-2002". Let`s see what is the right data.

In [ ]:
(database
 .groupby(["data"])
 .count()
 .sort_values(["group"])
 .query(" group != 20  ")
)

So the problem is in 01-Oct-2002 and one of then should be 01-Nov-2002.

In [ ]:
# The right data shod be '01-Nov-2002'
database = (database
            .assign(duplicated = lambda df:  df[["group", "data"]].duplicated(),
                    data       =  lambda df: df["data"].mask(df["duplicated"] == True, '01-Nov-2002')
                    )
            )
database.query(" duplicated == True ")

Since there are no duplicated data anymore, we can wide the database. But before that, to avoid numerical labels, we paste a "group_" suffix to the group values.

In [ ]:
# Now there are no duplicated data, we can wide the DataFrame
database_wide = (database
                        .assign(group = lambda df: "group_" + df["group"].astype("str") )
                        .pivot(index = ["data"],
                               columns = ["group"],
                               values = "y"
                               )
                        .reset_index()
                        .rename_axis(None, axis = 1)
                        )
database_wide.describe()

Now we can solve the outlier problem. We will do a median imputation.

In [218]:
limit = np.quantile(database_wide.drop(["data"], axis = 1), 0.999) # limit to mark as wrong value

database_new = (database_wide
                .assign(group_1_aux = lambda df: df["group_1"].mask( df["group_1"] > limit, np.nan),
                        group_1     = lambda df: df["group_1_aux"].fillna(df["group_1_aux"].median)
                        )
                .drop(["group_1_aux"], axis = 1)
                )
database_new.describe()

,group_10,group_11,group_12,group_13,group_14,group_15,group_16,group_17,group_18,group_19,group_2,group_20,group_3,group_4,group_5,group_6,group_7,group_8,group_9
count,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000
mean,10.048824,9.996743,10.011681,9.958684,9.967945,9.967287,10.045631,9.975825,10.014140,9.970744,10.017845,9.955394,9.997491,9.948381,10.007684,10.062900,10.023976,9.981162,9.991193
std,0.535920,0.485479,0.566335,0.513044,0.562966,0.540261,0.461217,0.495306,0.369347,0.509023,0.508128,0.628132,0.502479,0.516962,0.558968,0.587898,0.605233,0.502275,0.531928
min,8.045215,8.132061,7.319235,7.593635,7.891626,7.207311,8.508621,8.496057,8.481863,7.818055,7.843785,6.954679,7.764694,7.725457,7.082428,8.261903,7.898420,8.022692,7.656090
25%,9.844656,9.813506,9.818821,9.714076,9.703836,9.794658,9.850055,9.788434,9.840511,9.854594,9.851758,9.702203,9.812865,9.713000,9.820939,9.815503,9.830740,9.855542,9.805584
50%,9.996489,9.987516,10.013541,9.994251,9.959631,9.979454,10.025458,9.981834,10.005137,9.999646,9.993396,10.006072,10.024649,9.964654,10.001468,10.021673,10.009085,9.999617,9.993424
75%,10.235389,10.201939,10.250564,10.223290,10.181924,10.165320,10.218791,10.162658,10.159051,10.171975,10.162437,10.267470,10.211335,10.192324,10.208402,10.259789,10.238493,10.146274,10.182783
max,12.563542,11.777333,12.171259,12.116019,12.286089,11.894928,11.752984,12.096245,11.310129,11.785191,12.906505,12.250087,12.674424,11.668456,12.583304,13.311029,13.541790,12.414211,13.018325


# 3 EXPORT DATA

To finish, we export the new database as .csv and .xlsx, adding years and months variables to facilitate further manipulation. We also export a long version of the new data, that would be more suitable for some applications.

In [223]:
database_new = (database_new
                .assign(year = lambda df: df["data"].str.slice(7,11),
                        month = lambda df: df["data"].str.slice(3,6),
                        )
                )
database_new.head()

,data,group_1,group_10,group_11,group_12,group_13,group_14,group_15,group_16,group_17,...,group_20,group_3,group_4,group_5,group_6,group_7,group_8,group_9,year,month
0,01-Apr-2000,10.01135,9.987146,8.372398,11.204862,10.000080,9.957437,9.366857,10.019017,9.958579,...,9.816199,9.528391,9.682856,10.021460,10.634633,10.090673,10.032882,9.189847,2000,Apr
1,01-Apr-2001,10.77335,10.020888,10.025045,10.342786,9.386033,10.039840,9.895056,9.813690,9.903303,...,10.937522,9.729937,9.661809,9.998162,10.097819,9.961338,10.108989,10.224766,2001,Apr
2,01-Apr-2002,9.379179,8.486581,9.864623,9.487962,10.232624,11.681487,9.978239,9.611069,9.910267,...,10.814165,9.584221,10.889030,9.982543,9.813211,10.015782,9.535142,8.915741,2002,Apr
3,01-Apr-2003,10.017709,9.482085,10.433054,10.060909,9.379115,10.216647,9.792787,9.771448,9.804754,...,9.657993,10.567198,10.078124,10.175519,9.776505,9.105832,9.939589,10.259827,2003,Apr
4,01-Apr-2004,9.821178,9.959817,9.304840,10.553361,9.791334,9.876744,10.003894,9.879274,9.536919,...,10.040601,12.674424,10.354162,10.071010,9.904600,9.963596,10.352201,10.558290,2004,Apr


In [225]:
database_new.to_csv(Path(output_dir, "database_new.csv.zip"), sep = ",", decimal = ".")

(database_new
 .melt(id_vars = ["data", 'year', 'month'],
       value_vars = ['group_1', 'group_10', 'group_11', 'group_12', 'group_13',
                     'group_14', 'group_15', 'group_16', 'group_17', 'group_18', 'group_19',
                     'group_2', 'group_20', 'group_3', 'group_4', 'group_5', 'group_6',
                     'group_7', 'group_8', 'group_9'],
       var_name = "groups",
       value_name = "y"
       )
 .to_csv(Path(output_dir, "database_new_long.csv.zip"), sep = ",", decimal = ".")
 )
